In [1]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import accuracy_score

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [3]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [4]:

train_images = train_images / 255.0
test_images = test_images / 255.0

In [5]:

model = keras.Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])

In [6]:

model.compile(optimizer='adam',
              loss= SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [7]:

model.fit(train_images, train_labels, epochs=100)

Epoch 1/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.5026 - accuracy: 0.8243
Epoch 2/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3780 - accuracy: 0.8631
Epoch 3/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3373 - accuracy: 0.8766
Epoch 4/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.3131 - accuracy: 0.8852
Epoch 5/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2939 - accuracy: 0.8922
Epoch 6/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2776 - accuracy: 0.8980
Epoch 7/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2669 - accuracy: 0.9008
Epoch 8/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2553 - accuracy: 0.9050
Epoch 9/100
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2448 - accuracy: 0.9092
Epoch 10/100
1875/1875 [==============================] - 3s 1ms/step - l

In [10]:
KERAS_MODEL_NAME = "tf_model_fashion_mnist.h5"

In [13]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 0s - loss: 0.8107 - accuracy: 0.8844

Test accuracy: 0.8844000101089478


In [14]:

TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"

In [15]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp_nvupll0/assets


In [16]:
tflite_model_name = TF_LITE_MODEL_FILE_NAME
open(tflite_model_name, "wb").write(tflite_model)

408544

In [17]:

interpreter = tf.lite.Interpreter(model_path = TF_LITE_MODEL_FILE_NAME)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [ 1 28 28]
Input Type: <class 'numpy.float32'>
Output Shape: [ 1 10]
Output Type: <class 'numpy.float32'>


In [18]:
interpreter.resize_tensor_input(input_details[0]['index'], (10000, 28, 28))
interpreter.resize_tensor_input(output_details[0]['index'], (10000, 10))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [10000    28    28]
Input Type: <class 'numpy.float32'>
Output Shape: [10000    10]
Output Type: <class 'numpy.float32'>


In [19]:
test_imgs_numpy = np.array(test_images, dtype=np.float32)

In [20]:
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

Prediction results shape: (10000, 10)


In [21]:
acc = accuracy_score(prediction_classes, test_labels)

In [22]:


print('Test accuracy TFLITE model :', acc)

Test accuracy TFLITE model : 0.8844
